In [ ]:
# default_exp core
%load_ext lab_black

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import os

if os.name == "nt":
    git_exe = r"C:\Program Files\Git\cmd\git.exe"
else:
    git_exe = "/usr/bin/git"
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = git_exe
import sys
import shlex
import argparse
import pandas as pd
from git import Repo
from pathlib import Path
from collections import namedtuple
from subprocess import Popen, PIPE


python = sys.executable
home = Path(os.path.expanduser("~"))
blogslicer = Path(home / Path("github/blogslicer/blogslicer/core.py"))
if hasattr(__builtins__, "__IPYTHON__") or __name__ != "__main__":
    from IPython.display import display, Markdown

    h1 = lambda text: display(Markdown(f"# {text}"))
    h2 = lambda text: display(Markdown(f"## {text}"))
    h3 = lambda text: display(Markdown(f"### {text}"))

    file = "site.csv"
else:
    h1 = lambda text: print(f"# {text}")
    h2 = lambda text: print(f"## {text}")
    h3 = lambda text: print(f"## {text}")

    aparser = argparse.ArgumentParser()
    add_arg = aparser.add_argument
    add_arg("-f", "--file", required=True)
    args = aparser.parse_args()
    file = args.file

h1("Generaring sites...")
file_obj = Path(file)
df = pd.read_csv(file_obj, delimiter="|")
df = df.applymap(lambda x: x.strip())
df.columns = [x.strip() for x in df.columns]
Site = namedtuple("Site", "path, apex, title, gaid, tagline")


def git(cwd, args):
    cmd = [git_exe] + shlex.split(args)
    process = Popen(
        args=cmd,
        cwd=cwd,
        stdout=PIPE,
        stderr=PIPE,
        shell=False,
        bufsize=1,
        universal_newlines=True,
    )
    print(f"GIT: {shlex.join(cmd)}")
    for line in process.stdout:
        print(line.strip())
        sys.stdout.flush()


h2(f"Python: {python}")
h2(f"Blogslicer: {blogslicer}")
print()
for index, series in df.iterrows():
    site = Site(**series.to_dict())
    h3(site.apex)
    here = Path(home / site.path)
    [x.unlink() for x in Path(here / "_posts/").glob("*")]

    # Blog Slicer
    cmd = f'{python} {blogslicer} -p {here} -t "{site.title}" -s "blog" -a "Mike Levin"'
    print(cmd, end="\n\n")
    with Popen(args=cmd, cwd=here, stdout=PIPE, stderr=PIPE, shell=True) as pout:
        for line in pout.stdout.readlines():
            print(line.decode().strip())

    git(here, "add _posts/*")
    git(here, 'commit -am "Publising Blog Posts"')
    git(here, "push")


h2("Done!")

# Generaring sites...

## Python: C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe

## Blogslicer: C:\Users\mikle\github\blogslicer\blogslicer\core.py

### MikeLev.in

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\MikeLev.in -t "Mike Levin" -s "blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-17-post-139.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-16-post-138.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-16-post-137.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-15-post-136.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-15-post-135.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-15-post-134.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-15-post-133.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-14-post-132.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-14-post-131.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-14-post-130.md
## FILE: C:\Users\mikle\github\MikeLev.in/_posts/2022-06-13-post-129.md
## F

## Done!

In [ ]:
!nbdev_clean_nbs
!nbdev_build_lib

Converted 00_core.ipynb.
Converted index.ipynb.
